In [10]:
import sklearn.metrics
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
from collections import OrderedDict
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import  matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import numpy as np
import pandas as pd



In [11]:
def MAPE_regression(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 

In [12]:
#NuralNet (Regression)
class RegressionNet(nn.Module):
    def __init__(self, X_train, hiddenlayer, node, Ouputnode):
        super().__init__() #nn.Module의 __init__속성 및 메소드를 불러옴
        self.X_train = X_train
        self.hiddenlayer = hiddenlayer
        self.node = node
        self.Ouputnode = Ouputnode
        ordered_dict = OrderedDict()
        ordered_dict['Linear1'] = nn.Linear(X_train.size(1) , node)
        ordered_dict['relu1'] = nn.ReLU()
        for i in range(2 , hiddenlayer+2 ):
            ordered_dict['Linear{}'.format(i)] = nn.Linear(node, node)
            ordered_dict['relu{}'.format(i)] = nn.ReLU()
        ordered_dict['Linear{}'.format(hiddenlayer+2)] = nn.Linear(node, Ouputnode)
        self.layer1  = nn.Sequential(ordered_dict)
        
        
    def predict(self, X_train):
        pred_x = self.layer1(X_train)
        
        return pred_x
    
    def fit(self,epochs,loader, criterion, optimizer):
        losses = []
        for epoch in range(epochs):
            for step, (x, label) in enumerate(loader):
                optimizer.zero_grad()  # optimizer의 매개변수를 0으로 만듬
                y_pred = self.layer1(x)

                loss = criterion(y_pred, label)
                losses.append(loss)
                loss.backward()
                optimizer.step()
            if epoch % 10 == 0:
                print("{}epoch // loss ={}".format(epoch, loss))
                
   
        

In [13]:
#NuralNet (Classification)
class ClassificationNet(nn.Module):
    def __init__(self, X_train, hiddenlayer, node, Ouputnode):
        super().__init__() #nn.Module의 __init__속성 및 메소드를 불러옴
        self.X_train = X_train
        self.hiddenlayer = hiddenlayer
        self.node = node
        self.Ouputnode = Ouputnode
        ordered_dict = OrderedDict()
        ordered_dict['Linear1'] = nn.Linear(X_train.size(1) , node)
        ordered_dict['relu1'] = nn.ReLU()
        for i in range(2 , hiddenlayer+2 ):
            ordered_dict['Linear{}'.format(i)] = nn.Linear(node, node)
            ordered_dict['relu{}'.format(i)] = nn.ReLU()
        ordered_dict['Linear{}'.format(hiddenlayer+2)] = nn.Linear(node, Ouputnode)
        self.layer1  = nn.Sequential(ordered_dict)
        print(self.layer1)
        
    def predict(self, X_train):
        pred_x = self.layer1(X_train)
        
        return pred_x
    
    def fit(self,epochs,loader, criterion, optimizer):
        losses = []
        for epoch in range(epochs):
            for step, (x, label) in enumerate(loader):
                optimizer.zero_grad()  # optimizer의 매개변수를 0으로 만듬
                y_pred = self.layer1(x)

                loss = criterion(y_pred, label)
                losses.append(loss)
                loss.backward()
                optimizer.step()
            if epoch % 10 == 0:
                print("{}epoch // loss ={}".format(epoch, loss))

In [49]:

def CNet(X_train, y_train, X_test,y_test, hiddenlayer, node, Ouputnode, epochs):
    '''
    
        @Input 
            X_train : type : array
            y_train : type : array
            X_test : type : array
            hiddenlayer : type : int
            node : type : int
            Ouputnode : type : int
            epochs : type : int
            
        @Output
            pred_y_test : type : array
            
    '''
    
    
    # 데이터를 파이토치 텐서로 변경
    X = torch.Tensor(X_train.values)
    Y = torch.Tensor(y_train.values)
    X_test = torch.Tensor(X_test.values)
    
    # 학습을 위한 데이터 전처리 
    dataset = TensorDataset(X, Y)
    loader = DataLoader(dataset, batch_size=1, shuffle=True)
    
    #신경망 생성하기
    model = ClassificationNet(X, hiddenlayer, node, Ouputnode)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    model.fit(epochs,loader, criterion, optimizer)
    
    
    #학습되 모델 저장
    torch.save(model, "CNet.pt")
    
    #테스트 데이터로 예측 결과 반환
    pred_y_test = model.predict(X_test)
    
    #파이토치 텐서를 넘파이 어레이로 전환
    pred_y_test = pred_y_test.detach().numpy()
    
      # calculate AUC of model
    fpr, tpr, thresholds = roc_curve(y_test, pred_y_test)
    auc = roc_auc_score(y_test, pred_y_test)
      
    return  ({"auc_score": auc, "fpr":fpr, "tpr":tpr})
    

In [39]:
def RNet(X_train, y_train, X_test, y_test , hiddenlayer, node, Ouputnode, epochs):
    
    '''
    
        @Input 
            X_train : type : array
            y_train : type : array
            X_test : type : array
            hiddenlayer : type : int
            node : type : int
            Ouputnode : type : int
            epochs : type : int
            
        @Output
            pred_y_test : type : array
            
    '''
    
    # 데이터를 파이토치 텐서로 변경
    X = torch.Tensor(X_train.values)
    Y = torch.Tensor(y_train.values)
    X_test = torch.Tensor(X_test.values)
    
    # 학습을 위한 데이터 전처리 
    dataset = TensorDataset(X, Y)
    loader = DataLoader(dataset, batch_size=1, shuffle=True)

    # 신경망 생성하기
    model = RegressionNet(X, hiddenlayer, node, Ouputnode)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.fit(epochs,loader, criterion, optimizer)
            
    #학습되 모델 저장
    torch.save(model, "RNet.pt")
    
    #테스트 데이터로 예측 결과 반환
    pred_y_test = model.predict(X_test)
    
    #파이토치 텐서를 넘파이 어레이로 전환
    pred_y_test = pred_y_test.detach().numpy()
    
    MSE = mean_squared_error(y_test, pred_y_test)  # MSE 값은 출력하지 않아도 됩니다. RMSE를 구하기 위한 절차입니다.
    
    # 출력 대상입니다. 성능지표로써 3가지값이 출력되게 됩니다.
    RMSE = np.sqrt(MSE)
    MAPE = MAPE_regression(y_test, pred_y_test)
    R_squared = r2_score(y_test, pred_y_test)
    return {'pred_y' : pred_y_test, 'RMSE' : RMSE, 'MAPE' : MAPE, "R_squared" : R_squared}

In [40]:
df1 = pd.read_csv("ab_te.csv")
df2 = pd.read_csv("RMS_bearing.csv")

In [41]:
x = df1.iloc[:,[0, 1, 2, 3]]
y = df1.iloc[:,[4]]
x1 = df2.iloc[:,[0,1,2]]
y1 = df2.iloc[:,[3]]


X_train,X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=33)

X1_train,X1_test, y1_train, y1_test = train_test_split(x1,y1, test_size=0.2, random_state=33)

In [42]:
pred_y_test = CNet(X_train, y_train, X_test, 2, 5, 1, 10)

TypeError: CNet() missing 1 required positional argument: 'epochs'

In [43]:
print(pred_y_test)

tensor([[0.0555],
        [0.0562],
        [0.0568],
        [0.0699],
        [0.0684],
        [0.0575],
        [0.0563],
        [0.0556],
        [0.0707],
        [0.0575],
        [0.0557],
        [0.0560],
        [0.0556],
        [0.0566],
        [0.0560],
        [0.0564],
        [0.0560],
        [0.0616],
        [0.0563],
        [0.0609],
        [0.0655],
        [0.0560],
        [0.0563],
        [0.0558],
        [0.0574],
        [0.0590],
        [0.0561],
        [0.0558],
        [0.0558],
        [0.0567],
        [0.0602],
        [0.0586],
        [0.0565],
        [0.0714],
        [0.0250],
        [0.0621],
        [0.0556],
        [0.0566],
        [0.0622],
        [0.0564],
        [0.0558],
        [0.0661],
        [0.0560],
        [0.0560],
        [0.0556],
        [0.0562],
        [0.0616],
        [0.0561],
        [0.0564],
        [0.0611],
        [0.0588],
        [0.0597],
        [0.0587],
        [0.0557],
        [0.0668],
        [0

In [44]:
pred_y1_test = RNet(X1_train, y1_train, X1_test, y1_test, 2, 5, 1, 10)

0epoch // loss =4.4145748688606545e-05


In [46]:
RNET = torch.load("RNet.pt")
RNET.eval()

RegressionNet(
  (layer1): Sequential(
    (Linear1): Linear(in_features=3, out_features=5, bias=True)
    (relu1): ReLU()
    (Linear2): Linear(in_features=5, out_features=5, bias=True)
    (relu2): ReLU()
    (Linear3): Linear(in_features=5, out_features=5, bias=True)
    (relu3): ReLU()
    (Linear4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [47]:
CNET = torch.load("CNet.pt")
CNET.eval()

ClassificationNet(
  (layer1): Sequential(
    (Linear1): Linear(in_features=4, out_features=5, bias=True)
    (relu1): ReLU()
    (Linear2): Linear(in_features=5, out_features=5, bias=True)
    (relu2): ReLU()
    (Linear3): Linear(in_features=5, out_features=5, bias=True)
    (relu3): ReLU()
    (Linear4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [48]:
X_test = torch.Tensor(X_test.values)
pred_y_test = CNET.predict(X_test)
print(pred_y_test)

tensor([[-0.1013],
        [-0.1066],
        [-0.0729],
        [-0.0628],
        [-0.1021],
        [-0.0902],
        [-0.0893],
        [-0.1044],
        [-0.0548],
        [-0.0786],
        [-0.0996],
        [-0.0758],
        [-0.0576],
        [-0.0892],
        [-0.1011],
        [-0.0693],
        [-0.0749],
        [-0.0762],
        [-0.1049],
        [-0.0645],
        [-0.0915],
        [-0.0816],
        [-0.0663],
        [-0.0922],
        [-0.0608],
        [-0.0853],
        [-0.0695],
        [-0.0698],
        [-0.0787],
        [-0.0999],
        [-0.0945],
        [-0.0626],
        [-0.0829],
        [-0.0840],
        [-0.0659],
        [-0.0735],
        [-0.0968],
        [-0.0650],
        [-0.1009],
        [-0.0986],
        [-0.0753],
        [-0.0589],
        [-0.0964],
        [-0.0725],
        [-0.0634],
        [-0.0694],
        [-0.0702],
        [-0.0725],
        [-0.0662],
        [-0.0904],
        [-0.0758],
        [-0.0726],
        [-0.

In [37]:
X1_test = torch.Tensor(X1_test.values)
pred_y_test = RNET.predict(X1_test)
print(pred_y_test)

tensor([[0.0555],
        [0.0562],
        [0.0568],
        [0.0699],
        [0.0684],
        [0.0575],
        [0.0563],
        [0.0556],
        [0.0707],
        [0.0575],
        [0.0557],
        [0.0560],
        [0.0556],
        [0.0566],
        [0.0560],
        [0.0564],
        [0.0560],
        [0.0616],
        [0.0563],
        [0.0609],
        [0.0655],
        [0.0560],
        [0.0563],
        [0.0558],
        [0.0574],
        [0.0590],
        [0.0561],
        [0.0558],
        [0.0558],
        [0.0567],
        [0.0602],
        [0.0586],
        [0.0565],
        [0.0714],
        [0.0250],
        [0.0621],
        [0.0556],
        [0.0566],
        [0.0622],
        [0.0564],
        [0.0558],
        [0.0661],
        [0.0560],
        [0.0560],
        [0.0556],
        [0.0562],
        [0.0616],
        [0.0561],
        [0.0564],
        [0.0611],
        [0.0588],
        [0.0597],
        [0.0587],
        [0.0557],
        [0.0668],
        [0